In [1]:

from fsquery import FSQuery
import csv
import random

pages_path = "/home/phil/Documents/new_dev_tree/writing/thoughtstorms/pages/"
fsq = FSQuery(pages_path).Ext("md").NoFollow(".work").NoFollow(".git").FileOnly()

NO_SELECTIONS = 5


In [2]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
        
def filter_line(x) :
    if x == "" : return False
    if x == "\n" : return False
    if not x.strip() : return False
    if len(x) < 10 : return False
    if x[0:9] == "#redirect" : return False
    if x[0:6] == "*added" : return False
    if x[0:12] == "*originally*" : return False
    if "{=linkbin=}" in x : return False    

    if x[0:11] == "### linkbin" : return False
    if x[0:10] == "can't find" : return False

    return True

def remove_stops(x) :
    x = x.strip()
    xs = x.split(" ")
    xs = [x.lower() for x in xs if not x.lower() in ENGLISH_STOP_WORDS]
    return " ".join(xs)

def random_sample_of_lines(xs) :
    return [x for x in xs if random.randint(1,1000) < 500]

with open("out.csv",mode="w") as out :
    writer = csv.writer(out)

    writer.writerow(["Line","Page"])
    for n in fsq :
        name = (n.abs.split("/")[-1]).split(".")[0]
        f = n.open_file()
        lines = [remove_stops(x) for x in f.readlines() if filter_line(remove_stops(x))]
        
        for i in range(NO_SELECTIONS) :
            sample = " ".join(random_sample_of_lines(lines))
            writer.writerow([sample,name])


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("out.csv")
print df.head()

                                                Line       Page
0                                                NaN  2008crash
1  [https://www.theguardian.com/commentisfree/201...  2008crash
2  [https://www.theguardian.com/commentisfree/201...  2008crash
3                                                NaN  2008crash
4  [https://www.theguardian.com/commentisfree/201...  2008crash


In [38]:

from nltk.corpus import stopwords  
from sklearn.feature_extraction.text import CountVectorizer  

df = df[df.Line.notnull()]
texts = df["Line"].tolist()
pages = df["Page"].tolist()


vectorizer = CountVectorizer(decode_error='ignore',max_features=2500)  

X = vectorizer.fit_transform(texts).toarray() 

In [40]:
from sklearn.feature_extraction.text import TfidfTransformer  
tfidfconverter = TfidfTransformer()  
X = tfidfconverter.fit_transform(X).toarray()  

In [41]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, pages, test_size=0.2, random_state=0)  

['incaroadnetworks',
 'thesanctuarycafe',
 'nonplace',
 'oncontainerization',
 'balanceofpower',
 'resilienceprojects',
 'looseaccounting',
 'individualrecognitionreadinglist',
 'networksandmusicalecology',
 'englandshiddenreverse',
 'doublecomma',
 'simulationresults',
 'hypersemantics',
 'dissipativestructure',
 'douglasrushkoff',
 'socialinterfaces',
 'politicalblogosphere',
 'memeengine',
 'alwaysonpanopticon',
 'spreadsheets',
 'todo',
 'anotherantintheswarm',
 'ericdrexler',
 'rubylanguage',
 'gasolinewarfare',
 'softwaresupplychain',
 'liberalbias',
 'deliciouslibrary',
 'energyweb',
 'languagesasnetworks',
 'buysidepublishing',
 'flatland',
 'lastfm',
 'onpatterns',
 'automation-abstractionempowered',
 'classwarbetweencityandcountry',
 'typedthreadeddiscussion',
 'agroindustrialautocatalyst',
 'javavirtualmachine',
 'gardenimagologies',
 'nonzero',
 'altoparaiso',
 'corporateladder',
 'hackerpolitics',
 'tcllanguage',
 'internetplayas',
 'creativemistyping',
 'otherhotwars',
 '

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(X_train, y_train)  

In [ ]:
y_pred = classifier.predict(X_test)  